<h1 style="text-align:center; color:black; text-decoration:underline red;">Cartographie interactive des résultats</h1>

In [ ]:
import pandas
pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_columns", None)
import os
os.environ["USE_PYGEOS"] = "0"
import geopandas
import folium

Ce notebook a pour but de présenter les résultats de la préparation de données, sous forme de carte interactive folium. Un premier résultat est proposé grâce au logiciel QGIS :
<br/>
<img width=800px height=auto src="..\\RESULTATS\\carto.png"/>

Bien que la carte puisse informer, on peut suggérer une carte interactive :
- Pour afficher les données exactes,
- Ajouter plus de couches, pour pouvoir mieux contextualiser les arrêts de bus,
- Ajouter des info-bulles pour avoir plus d'information sur la commune,
- Obtenir plus de contexte géographique, notamment avec un fond de carte tel qu'ESRI Satellite, par exemple.

In [ ]:
# Ouvrir les données, et les reprojeter en degré pour Folium.
contours_vdo = geopandas.read_file(filename=r"..\TRAITEMENT\communes_valdoise_2022.geojson",
                                   dtype={"contours_vdo":str})
contours_vdo.to_crs(4326, inplace=True)

arrets_bus = geopandas.read_file(filename=r"..\TRAITEMENT\arrets_bus_vdo.geojson")
arrets_bus.to_crs(4326, inplace=True)

pop_21 = pandas.read_csv(r"..\TRAITEMENT\population_2021.csv", 
                         dtype={"COM":str},
                         sep=",", decimal=".", encoding="utf-8")
pop_21.head()

In [ ]:
# Compter le nombre d'arrêt de bus dans les communes #

pop_21.head() #Champ de jointure identifié : COM
contours_vdo.head() #Champ de jointure identifié : insee

contours_vdo["NB_BUS"] = 0

for i, r in contours_vdo.iterrows() :
    com = r["geometry"]

    for j, k in arrets_bus.iterrows() :

        if arrets_bus.loc[j, "geometry"].within(other=com) == True :
            contours_vdo.loc[i, "NB_BUS"] = contours_vdo.loc[i, "NB_BUS"] + 1
        else : 
            pass

# Faire la jointure #

contours_vdo = pandas.merge(left=contours_vdo, right=pop_21, left_on="insee", right_on="COM", how="left", validate="1:1")
contours_vdo["PART_POP_VDO"] = contours_vdo["PTOT"].apply(func= lambda x : ( (x / contours_vdo["PTOT"].sum()) * 100))
contours_vdo.head()


In [ ]:
bus_vdo = contours_vdo.copy(deep=True)
bus_vdo.to_crs(3857, inplace=True)
bus_vdo["geometry"] = bus_vdo["geometry"].centroid
bus_vdo.to_crs(4326, inplace=True)

bus_vdo.head()

In [ ]:
#Centre de la carte. Nous travaillons sur l'IDF et précisément sur le Val d'Oise.
#Nous allons donc centrer la carte sur le Val d'Oise.

#On commence par dissoudre les communes du même département.
diss = contours_vdo.dissolve(by="numdep")

m = folium.Map(
    location = (diss.centroid.y, diss.centroid.x),
    zoom_start = 10,
    min_zoom = 10,
    control_scale = True, #Ajouter une échelle.
)

folium.GeoJson(
    data = contours_vdo,
    name = "Bordure commune",
    style_function = lambda feature : {
        "fillColor" : "#00000000",
        "color" : "black",
        "weight" : 2
    }
    ).add_to(m)

contours_vdo.explore(
    m = m,
    column = "PART_POP_VDO",
    scheme = "naturalbreaks",
    cmap = "Reds",
    color = "black",
    legend = True,
    tooltip = False,
    popup = ["nomcom", "PART_POP_VDO"],
    name = "Part de la population communale dans la population départementale")

contours_vdo.explore(
    m = m,
    column = "aire",
    scheme = "naturalbreaks",
    cmap = "Blues",
    legend = True,
    tooltip = False,
    popup = ["nomcom", "aire"],
    name = "Surface de la commune en m2")

for i, r in bus_vdo[ ["nomcom", "NB_BUS", "geometry"] ].iterrows() :

    folium.Circle(
            location = [ r["geometry"].y, r["geometry"].x ],
            popup = '%s (%.1f)' % (r["nomcom"], r["NB_BUS"]),
            radius = r["NB_BUS"] * 10,
            color='crimson',
            fill=True,
            fill_color='crimson',
            name = "Nombre d'arrêt de bus unique"
        ).add_to(m)
    
folium.TileLayer("https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
                 attr = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community",
                 name = "ESRI Satellite"
                 ).add_to(m)

folium.LayerControl().add_to(m)


m.save(outfile = r"..\RESULTATS\carte_interactive_resultat.html")

m #Afficher la carte
